<a href="https://colab.research.google.com/github/evelynmitchell/Presentations/blob/main/TrioAsync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trio Easier Async for Python

Why? 

Batteries included.

Obvious. All async function are named that way.

Tracing, Cancellation, Timeouts, IO handlers, Testing...

Scalable.

Doesn't do to much. Leaves choices up to callers where appropriate.

https://trio.readthedocs.io/en/stable/

## Install

In [1]:
!python3 -m pip install -U trio

     |████████████████████████████████| 358kB 5.1MB/s 


## Tutorial

Welcome to the Trio tutorial! Trio is a modern Python library for writing asynchronous applications – that is, programs that want to do multiple things at the same time with parallelized I/O, like a web spider that fetches lots of pages in parallel, a web server juggling lots of simultaneous downloads… that sort of thing. Here we’ll try to give a gentle introduction to asynchronous programming with Trio.


In [2]:
# A regular function
def regular_double(x):
    return 2 * x

# An async function
async def async_double(x):
    return 2 * x


“Async” is short for “asynchronous”; we’ll sometimes refer to regular functions like regular_double as “synchronous functions”, to distinguish them from async functions.

You can only use the **await** keyword in **async functions**.

In [3]:
async def print_double(x):
    print(await async_double(x))   # <-- OK!

Just calling an async function won't do what you expect.

In [4]:
print_double(9)

<coroutine object print_double at 0x7f3ba2e1d8c0>

You need to write a **runner** function to call the **async** function.

In [5]:
import trio

async def async_double(x):
    return 2 * x

trio.run(async_double, 3)  # returns 6

/usr/local/lib/python3.7/dist-packages/trio/_core/_multierror.py:464: RuntimeWarning: IPython detected, but you already have a custom exception handler installed. I'll skip installing Trio's custom handler, but this means MultiErrors will not show full tracebacks.
  category=RuntimeWarning,


6

**Async** functions are useful for **IO** which is often waiting for something to return, or for a person to do something. So, trio has a lot of builtin functions for doing IO.

In [6]:
import trio

async def double_sleep(x):
    await trio.sleep(2 * x)

trio.run(double_sleep, 3)  # does nothing for 6 seconds then returns

This suggests an architecture pattern called an **async sandwich**
```
trio.run -> double_sleep -> trio.sleep
```

You wrap async functions in a runner and a handler, both of which are async. So the runner calls the async function which then calls the final async function.

Then, both the runner and the wrapper can wait on the world, and the function being called doesn't have to change.

```
trio.run -> [async function] -> ... -> [async function] -> trio.whatever
```
Is the general pattern.

If you forget to **await** you'll get a warning not an error, and the code will complete.

```
 RuntimeWarning: coroutine 'sleep' was never awaited
```

In [7]:
import time
import trio

async def broken_double_sleep(x):
    print("*yawn* Going to sleep")
    start_time = time.perf_counter()

    # Whoops, we forgot the 'await'!
    trio.sleep(2 * x)

    sleep_time = time.perf_counter() - start_time
    print(f"Woke up after {sleep_time:.2f} seconds, feeling well rested!")

trio.run(broken_double_sleep, 3)

*yawn* Going to sleep
Woke up after 0.00 seconds, feeling well rested!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: coroutine 'sleep' was never awaited
  if __name__ == '__main__':


"Forgetting an await like this is an incredibly common mistake. You will mess this up. Everyone does. And Python will not help you as much as you’d hope 😞. The key thing to remember is: if you see the magic words **RuntimeWarning: coroutine '...' was never awaited**, then this always means that you made the mistake of leaving out an await somewhere, and you should ignore all the other error messages you see and go fix that first, because there’s a good chance the other stuff is just collateral damage. I’m not even sure what all that other junk in the PyPy output is. Fortunately I don’t need to know, I just need to fix my function!"

## Running multiple async functions at the same time

In [8]:
# tasks-intro.py

import trio

async def child1():
    print("  child1: started! sleeping now...")
    await trio.sleep(3)
    print("  child1: exiting!")

async def child2():
    print("  child2: started! sleeping now...")
    await trio.sleep(1)
    print("  child2: exiting!")

async def parent():
    print("parent: started!")
    async with trio.open_nursery() as nursery:
        print("parent: spawning child1...")
        nursery.start_soon(child1)

        print("parent: spawning child2...")
        nursery.start_soon(child2)

        print("parent: waiting for children to finish...")
        # -- we exit the nursery block here --
    print("parent: all done!")

trio.run(parent)

parent: started!
parent: spawning child1...
parent: spawning child2...
parent: waiting for children to finish...
  child2: started! sleeping now...
  child1: started! sleeping now...
  child2: exiting!
  child1: exiting!
parent: all done!


This example shows the use of **Async with** to group **Async** functions under a parent, so they are are handled within the same **with** or similarly a **for** statment. This pattern is called a **nursery** in Trio. The async funtions have similar lifetimes under the parent.

"There are only 4 lines of code that really do anything here. On line 17, we use trio.open_nursery() to get a “nursery” object, and then inside the async with block we call nursery.start_soon twice, on lines 19 and 22. There are actually two ways to call an async function: the first one is the one we already saw, using await async_fn(); the new one is nursery.start_soon(async_fn): it asks Trio to start running this async function, but then returns immediately without waiting for the function to finish. So after our two calls to nursery.start_soon, child1 and child2 are now running in the background. And then at line 25, the commented line, we hit the end of the async with block, and the nursery’s __aexit__ function runs. What this does is force parent to stop here and wait for all the children in the nursery to exit. This is why you have to use async with to get a nursery: it gives us a way to make sure that the child calls can’t run away and get lost. One reason this is important is that if there’s a bug or other problem in one of the children, and it raises an exception, then it lets us propagate that exception into the parent; in many other frameworks, exceptions like this are just discarded. **Trio never discards exceptions.**"

## Tracing 

In [10]:
class Tracer(trio.abc.Instrument):
    def before_run(self):
        print("!!! run started")

    def _print_with_task(self, msg, task):
        # repr(task) is perhaps more useful than task.name in general,
        # but in context of a tutorial the extra noise is unhelpful.
        print(f"{msg}: {task.name}")

    def task_spawned(self, task):
        self._print_with_task("### new task spawned", task)

    def task_scheduled(self, task):
        self._print_with_task("### task scheduled", task)

    def before_task_step(self, task):
        self._print_with_task(">>> about to run one step of task", task)

    def after_task_step(self, task):
        self._print_with_task("<<< task step finished", task)

    def task_exited(self, task):
        self._print_with_task("### task exited", task)

    def before_io_wait(self, timeout):
        if timeout:
            print(f"### waiting for I/O for up to {timeout} seconds")
        else:
            print("### doing a quick check for I/O")
        self._sleep_time = trio.current_time()

    def after_io_wait(self, timeout):
        duration = trio.current_time() - self._sleep_time
        print(f"### finished I/O check (took {duration} seconds)")

    def after_run(self):
        print("!!! run finished")

## And then use the tracer

In [11]:
trio.run(parent, instruments=[Tracer()])

!!! run started
### new task spawned: <init>
### task scheduled: <init>
### doing a quick check for I/O
### finished I/O check (took 0.002314217999810353 seconds)
>>> about to run one step of task: <init>
### new task spawned: __main__.parent
### task scheduled: __main__.parent
### new task spawned: <TrioToken.run_sync_soon task>
### task scheduled: <TrioToken.run_sync_soon task>
<<< task step finished: <init>
### doing a quick check for I/O
### finished I/O check (took 1.730700023472309e-05 seconds)
>>> about to run one step of task: <TrioToken.run_sync_soon task>
<<< task step finished: <TrioToken.run_sync_soon task>
>>> about to run one step of task: __main__.parent
parent: started!
parent: spawning child1...
### new task spawned: __main__.child1
### task scheduled: __main__.child1
parent: spawning child2...
### new task spawned: __main__.child2
### task scheduled: __main__.child2
parent: waiting for children to finish...
<<< task step finished: __main__.parent
### doing a quick che

## Trio and builtin Async

They don't play well together.


## Scale
"Excellent scalability: Trio can run 10,000+ tasks simultaneously without breaking a sweat, so long as their total CPU demands don’t exceed what a single core can provide. (This is common in, for example, network servers that have lots of clients connected, but only a few active at any given time.)"

## Cancellation
https://trio.readthedocs.io/en/stable/reference-core.html#cancellation

Timeouts
```
with trio.move_on_after(30):
    result = await do_http_get("https://...")
    print("result is", result)
print("with block finished")
```

move_on_after creates **Cancel Scope**

### Handling Cancellations
"Pretty much any code you write using Trio needs to have some strategy to handle Cancelled exceptions – even if you didn’t set a timeout, then your caller might (and probably will).

You can catch Cancelled, but you shouldn’t! Or more precisely, if you do catch it, then you should do some cleanup and then re-raise it or otherwise let it continue propagating (unless you encounter an error, in which case it’s OK to let that propagate instead). To help remind you of this fact, Cancelled inherits from BaseException, like KeyboardInterrupt and SystemExit do, so that it won’t be caught by catch-all except Exception: blocks."

"If you have a task that has to do a lot of work without any I/O, then you can use await sleep(0) to insert an explicit cancel+schedule point."

Let the caller handle timeouts.

When a scope is Cancelled, it will continue to raise Cancellation exceptions till handled.

Documentation patch needed

"When things go wrong: timeouts, cancellation and exceptions in concurrent tasks
TODO: give an example using fail_after()

TODO: explain Cancelled

TODO: explain how cancellation is also used when one child raises an exception

TODO: show an example MultiError traceback and walk through its structure

TODO: maybe a brief discussion of KeyboardInterrupt handling?"

### fail_after
Creates a cancel scope with the given timeout, and raises an error if it is actually cancelled.

Contributing

https://github.com/python-trio

This software is made available under the terms of *either* of the
licenses found in LICENSE.APACHE2 or LICENSE.MIT. Contributions to
Trio are made under the terms of *both* these licenses.